# Developing Activity class
__Keith Cheveralls__<br>
__March 2019__



In [ ]:
import os
import re
import sys
import gzip
import time
import pickle
import datetime
import numpy as np
from scipy import stats
import pandas as pd

import fitparse
from fitparse import FitFile
from matplotlib import pyplot as plt

%matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append('../')
import cypy2

root = '/home/keith/Downloads/export_7989839-1'

wahoo_example = '2326365683.fit.gz'
garmin_example = '2122584483.fit.gz'
garmin_indoor_example = '2324139976.fit.gz'

### Load a single activity directly from a FIT file

In [ ]:
a = cypy2.Activity.from_fit(os.path.join(root, 'activities', garmin_example))

In [ ]:
a.metadata

### Load all activities from cached strava export

In [ ]:
m = cypy2.StravaExportManager(root, from_cache=True)

In [ ]:
activities = []
for ind, d in enumerate(m.parsed_data):
    activities.append(cypy2.Activity.from_strava_export(d))

In [ ]:
[(ind, a.metadata) for ind, a in enumerate(activities) if a.metadata['device_model']=='elemnt'][-1]

In [ ]:
cypy2.Activity.id_from_fit(file_id=pd.Series(dict(time_created='2018-01-02 3:45:56')).apply(pd.to_datetime))

In [ ]:
cypy2.Activity.id_from_fit(file_id=pd.Series(dict(time_created='2018-01-02')).apply(pd.to_datetime))

In [ ]:
md = pd.DataFrame([a.metadata for a in activities])

In [ ]:
pd.concat((md.groupby('device_model').count().activity_id, md.groupby('device_model').sum()), axis=1)

In [ ]:
# rides with power but without heart rate
md.loc[(md.device_model=='edge520') & ~(md.heart_rate_flag) & (md.power_flag)]
# len(ids_power_wo_hrm)

In [ ]:
# rides without an antplus_device_type column in device_info
# these *should* be only rides without any sensors
ids_wo_antplus = []
for a in activities:
    if a.metadata['activity_type']=='ride':
        if 'antplus_device_type' not in a._fit_data['device_info'].columns:
            if 'heart_rate' in a._fit_data['record'].columns:
                ids_wo_antplus.append(a.metadata['activity_id'])
            
len(ids_wo_antplus)

In [ ]:
a = [a for a in activities if a.metadata['activity_id'].startswith('201604')][-1]

In [ ]:
a.metadata

In [ ]:
# cat all messages for some activities
dcat = pd.concat(tuple([a._fit_data['device_info'] for a in activities if a.metadata['device_model']=='edge520']))